In [2]:
import os

import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import gseapy as gp
from notebooks.tasks58.plots import plot_correlations, all_negative, all_positive
from notebooks.tasks58.scaling import median_scaling

import os

while os.getcwd()[-len('mcrc-cetuximab-analysis'):] != 'mcrc-cetuximab-analysis':
    os.chdir('..')
os.getcwd() 

'/home/max/mcrc-cetuximab-analysis'

In [7]:
tpms = pd.read_csv('raw/exprs_PRJNA805525.csv', index_col=0)
ann = pd.read_csv('raw/ann_PRJNA805525.csv', index_col=0)

In [ ]:

log_tpms_with_emt_signatures = log_tpms.copy()

ssgsea_results = gp.ssgsea(data=log_tpms.T,
                           gene_sets=emt_gene_lists,
                           outdir=None,
                           sample_norm_method='rank',
                           permutation_num=0,
                           no_plot=True,
                           min_size=0)

ssgsea_df = ssgsea_results.res2d.rename(columns={'Name': 'sample_id'})

for signature in emt_gene_lists:
    add = ssgsea_df.loc[ssgsea_df['Term'] == signature, ['sample_id', 'ES']]
    log_tpms_with_emt_signatures = log_tpms_with_emt_signatures.merge(add, left_index=True, right_on='sample_id').rename(columns={'ES': signature}).set_index('sample_id').astype({signature: np.float64})

print(log_tpms_with_emt_signatures.isna().sum().max())

log_tpms_with_emt_signatures_with_ann = log_tpms_with_emt_signatures.merge(ann, left_index=True, right_index=True)

df_post = log_tpms_with_emt_signatures_with_ann[log_tpms_with_emt_signatures_with_ann['time point'] == 'post-treatment']
df_pre = log_tpms_with_emt_signatures_with_ann[log_tpms_with_emt_signatures_with_ann['time point'] == 'pre-treatment']

# Scale separetely
for i, signature in enumerate(emt_gene_lists):
    df_pre[signature + '_rank'] = df_pre[signature].rank(ascending=False)
    df_pre[signature + '_med'] = median_scaling(df_pre[signature])
    df_post[signature + '_rank'] = df_post[signature].rank(ascending=False)
    df_post[signature + '_med'] = median_scaling(df_post[signature])

df_pre_pd = df_pre[df_pre['treatment response'] == 'PD']
df_pre_npd = df_pre[df_pre['treatment response'] == 'non-PD']
df_post_pd = df_post[df_post['treatment response'] == 'PD']
df_post_npd = df_post[df_post['treatment response'] == 'non-PD']

df_pre.to_csv('data/pre_ssgsea_expr_ann.csv')
df_post.to_csv('data/post_ssgsea_expr_ann.csv')